In [2]:
import ee
import geemap.eefolium as geemap
ee.Authenticate()

Enter verification code:  4/1AX4XfWhgp1uKGMvNElfacJINNxbQTIXna3TkL_vaIciHdHzuDtL8xyTdAhI



Successfully saved authorization token.


In [15]:
ee.Initialize()

In [16]:
zone = ee.FeatureCollection("users/emanuelgarciao/pnatural_23030")

In [17]:
def get_NDVI (img):
    return img.addBands (img.normalizedDifference(["B4","B3"]).rename("NDVI"))

In [18]:
m = geemap.Map()
m.centerObject(0)
vis_params={"min": 0 }
m

In [19]:
l7collection = (
    ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
    .filterBounds(zone)
    .filterDate("1999-01-01", "1999-12-31")
)

l7ndvi = l7collection.map(get_NDVI)

composite = l7ndvi.qualityMosaic("NDVI").clip(zone)

band = composite.select("NDVI")

In [20]:
m = geemap.Map()

m.centerObject(zone, zoom=10)

vis_params = {"min": 0, "max": 1, "palette": ["beb261", "8ffff9", "1cd810"]}

m.addLayer(band, vis_params, "NDVI")

m

In [21]:
bands = []

#  Since we will create a yearly NDVI image, we should create a loop that interates the years
years = range(2000, 2021)
for year in years:
    # We will use the product called LANDSAT/LE07/C01/T1_SR from Landsat 7 to compute NDVI
    # We will also filter all the images by area (using filterBounds)
    # and by year (using filterDate)
    l7collection = (
        ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
        .filterBounds(zone)
        .filterDate(str(year) + "01-01", str(year) + "12-31")
    )

    # The funciton map() is a very simple way to create loops in GEE
    # It allows to "project" a function (getNDVI) over a collection (l7collection)
    l7ndvi = l7collection.map(get_NDVI)

    # Compositing is a very common process in remote sensing that refers to the process 
    # of combining spatially overlapping images into a single one based on an 
    # aggregation function
    # In this case, we will creates a single image that contains the maximum NDVI
    # value per pixel using qualityMosaic function
    # I.e. The maximum value of NDVI in the pixel A could occurr in the date x,
    # while that maximum value in pixel B could occurr in date y
    # This means that the date of the image would be a composite of all the combined images
    composite = l7ndvi.qualityMosaic("NDVI").clip(zone)
    
    band = composite.select("NDVI")
    
    # Append the band to the list of all bands
    bands.append(band)
    

In [22]:
m = geemap.Map()

m.centerObject(zone, zoom=10)

vis_params = {"min": 0, "max": 1, "palette": ["beb261", "8ffff9", "1cd810"]}

for posicion, banda in enumerate(bands):
    m.addLayer(banda, vis_params, "NDVI_" + str(2000 + posicion))

m.addLayerControl()

m

EEException: DateRange: Bad date/time '200012-31'.

In [ ]:
data = []
for band in bands:
    data.append(band.redu)